# Identify Contrails with Keras

In [102]:
# reinstall tensorflow-io
# to avoid the UserWarning: unable to load libtensorflow_io_plugins.so

#!pip install tensorflow-io

In [103]:
%pwd

'/kaggle/working'

In [104]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pathlib
import random
import shutil

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [105]:
# This notebook is inspired by:

# Visualization:
# - https://www.kaggle.com/code/inversion/visualizing-contrails
# - https://www.kaggle.com/code/pranavnadimpali/comprehensive-eda-submission

# Models:
# - https://keras.io/examples/vision/oxford_pets_image_segmentation/
# - https://www.kaggle.com/code/shashwatraman/simple-unet-baseline-train-lb-0-580

In [106]:
#---------------------------------------------------------------------------79

## Setup

In [107]:
class ABI:
    bands = {name: idx for idx, name in enumerate([
        '08', '09', '10', '11', '12', '13', '14', '15', '16'])}
    colors = {name: idx for idx, name in enumerate([
        'red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'yellow', 'black'])}

In [108]:
N_TIMES_BEFORE = 4
N_TIMES_AFTER = 3

In [109]:
WORK_DIR = '/kaggle/working'  # preserved if notebook is saved
TEMP_DIR = '/kaggle/temp'  # just during current session

DATA_DIR = '/kaggle/input/google-research-identify-contrails-reduce-global-warming'

class Paths:
    train = os.path.join(DATA_DIR, 'train')
    valid = os.path.join(DATA_DIR, 'validation')


## Data Analysis and Visualization

In [110]:
DRAW = False

In [111]:
train_ids = os.listdir(Paths.train)
valid_ids = os.listdir(Paths.valid)
print(len(train_ids), len(valid_ids))

20529 1856


In [112]:
pixel_mask = np.load(os.path.join(DATA_DIR, 'train', train_ids[3], 'human_pixel_masks.npy'))
print(pixel_mask.shape)
print(pixel_mask.min(), pixel_mask.max())

(256, 256, 1)
0 1


In [113]:
sample_ids = train_ids

In [114]:
def plot_bands_over_time(sample_id, split_dir):
    """
    
    Adapted from:
    https://www.kaggle.com/code/pranavnadimpali/comprehensive-eda-submission
    
    Args: 
        sample_id(str): The id of the example i.e. '1000216489776414077'
        split_dir(str): The split directoryu i.e. 'test', 'train', 'val'
    """
    fig, axs = plt.subplots(8, len(ABI.bands), figsize=(16, 16)) 

    for band, j in ABI.bands.items():
        img = np.load(DATA_DIR + f"/{split_dir}/{sample_id}/band_{band}.npy")
        for i in range(8):
            axs[i, j].imshow(img[..., i]) 
            axs[i, j].set_title(f"Band {band}\nTime Step {i+1}") 

    plt.tight_layout()  
    plt.show()
    
if DRAW:
    plot_bands_over_time(sample_ids[3], 'train')

In [115]:
def normalize_range(data, bounds):
    """Maps data to the range [0, 1]."""
    return (data - bounds[0]) / (bounds[1] - bounds[0])

_T11_BOUNDS = (243, 303)
_CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
_TDIFF_BOUNDS = (-4, 2)

def get_ash_colors(sample_id, split_dir):
    """
    Based on bands: 11, 14, 15
    
    Args:
        sample_id(str): The id of the example i.e. '1000216489776414077'
        split_dir(str): The split directoryu i.e. 'test', 'train', 'val'
    """
    band15 = np.load(DATA_DIR + f"/{split_dir}/{sample_id}/band_15.npy")
    band14 = np.load(DATA_DIR + f"/{split_dir}/{sample_id}/band_14.npy")
    band11 = np.load(DATA_DIR + f"/{split_dir}/{sample_id}/band_11.npy")

    r = normalize_range(band15 - band14, _TDIFF_BOUNDS)
    g = normalize_range(band14 - band11, _CLOUD_TOP_TDIFF_BOUNDS)
    b = normalize_range(band14, _T11_BOUNDS)
    ash_colors = np.clip(np.stack([r, g, b], axis=2), 0, 1)
    
    return ash_colors

In [116]:
def get_pixel_mask(sample_id, split_dir):
    masks_path = DATA_DIR + f"/{split_dir}/{sample_id}/human_pixel_masks.npy"
    pixel_mask = np.load(masks_path)
    return pixel_mask

In [117]:
def plot_ash_colors(sample_id, split_dir, plot, time_step=4):

    ash_colors = get_ash_colors(sample_id, split_dir)
    img = ash_colors[..., time_step] # 5th image corresponds to ground truth
    
    ground_truth = get_pixel_mask(sample_id, split_dir)
    
    if plot:
        fig, axs = plt.subplots(1, 3, figsize=(16, 8))

        axs[0].imshow(img)
        axs[0].set_title("Ash Color Image")

        axs[1].imshow(ground_truth)
        axs[1].set_title("Ground Truth")

        axs[2].imshow(img)
        axs[2].imshow(ground_truth, cmap='Reds', alpha=.3, interpolation='none')
        axs[2].set_title('Contrail mask on ash color image')


        plt.tight_layout() 
        plt.show()

    return img
    
if DRAW:
    plot_ash_colors(sample_ids[3], 'train', True)

In [118]:
def plot_three_bands(sample_id, split_dir, bands, timestep=4):
    """
    
    Adapted from:
    https://www.kaggle.com/code/pranavnadimpali/comprehensive-eda-submission
    
    Args: 
        sample_id(str): The id of the example i.e. '1000216489776414077'
        split_dir(str): The split directoryu i.e. 'test', 'train', 'val'
    """
    fig, axs = plt.subplots(1, 3, figsize=(16, 16)) 

    for j, band in enumerate(bands):
        img = np.load(DATA_DIR + f"/{split_dir}/{sample_id}/band_{band}.npy")
        axs[j].imshow(img[..., timestep]) 
        axs[j].set_title(f"Band {band}\nTime Step {timestep+1}") 

    plt.tight_layout()  
    plt.show()
    
if DRAW:
    plot_three_bands(sample_ids[3], 'train', ['11', '14', '15'])
    plot_three_bands(sample_ids[3], 'train', ['08', '09', '10'])
    plot_three_bands(sample_ids[3], 'train', ['12', '13', '16'])

## Preprocessing

In [119]:
ash_path = pathlib.Path(os.path.join(TEMP_DIR, 'ash_colors_images'))
ash_path.mkdir(exist_ok=True, parents=True)

ash_path.resolve()  # get absolute path

PosixPath('/kaggle/temp/ash_colors_images')

In [120]:
#shutil.rmtree(ash_path)

In [121]:
split_dir = 'train'

# 1 images take about 3 MB with all 8 timesteps. 
# train set has 20529 samples, i.e. it would take 60 GB.

if False:

    print('convert to ash colors')
    for sample_id in tqdm(train_ids[:100]):

        ash_colors = get_ash_colors(sample_id, split_dir)
        #pixel_mask = get_pixel_mask(sample_id, split_dir)
    
    # 59.68it/s
    # 1 s/sample

if False:
    print('convert to ash colors and write')
    for sample_id in tqdm(train_ids[:100]):

        ash_colors = get_ash_colors(sample_id, split_dir)
        #pixel_mask = get_pixel_mask(sample_id, split_dir)

        ash_colors = ash_colors.astype(np.float16)

        image_path = ash_path/f"{sample_id}.npy"
        np.save(str(image_path), ash_colors)
        
    # 21.64it/s
    # 3 s/sample
        
if False:
    print('read')
    for sample_id in tqdm(train_ids[:100]):

        image_path = ash_path/f"{sample_id}.npy"
        np.load(str(image_path))
    
    # 634.78it/s
    # 0.1 s/sample
    # But is this true or are the files still cached by the OS?

## Build model

In [147]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as backend

In [165]:
SEED = 42

In [166]:
class Config:
    
    img_size = (256, 256)
    
    train = True
    
    num_epochs = 10
    num_classes = 1
    batch_size = 32
    
    seed = SEED
    

In [167]:
# https://keras.io/examples/keras_recipes/reproducibility_recipes/

# Set the seed using keras.utils.set_random_seed. This will set:
# 1) `numpy` seed
# 2) `tensorflow` random seed
# 3) `python` random seed
keras.utils.set_random_seed(Config.seed)

# See also:
# tf.config.experimental.enable_op_determinism()

Following U-Net model is adapted from: https://keras.io/examples/vision/oxford_pets_image_segmentation

In [168]:
def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model


# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

# Build model
model = get_model(Config.img_size, Config.num_classes)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 128, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

## Train model

In [169]:
class AshColorSingleFrames(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, sample_ids, split_dir):
        self.batch_size = batch_size
        self.img_size = img_size
        self.split_dir = split_dir
        self.sample_ids = sample_ids

    def __len__(self):
        return len(self.sample_ids) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_sample_ids = self.sample_ids[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, sample_id in enumerate(batch_sample_ids):
            img = get_ash_colors(sample_id, self.split_dir)
            x[j] = img[..., N_TIMES_BEFORE]
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")
        for j, sample_id in enumerate(batch_sample_ids):
            img = get_pixel_mask(sample_id, self.split_dir)
            y[j] = img
        return x, y

In [170]:
train_set = AshColorSingleFrames(4, Config.img_size, train_ids, 'train')
print('number of batches:', len(train_set))

number of batches: 5132


In [171]:
valid_set = AshColorSingleFrames(4, Config.img_size, valid_ids, 'valid')
print('number of batches:', len(valid_set))

number of batches: 464


Check batch dimensions (x, y):

In [172]:
train_set[0][0].shape, train_set[0][1].shape

((4, 256, 256, 3), (4, 256, 256, 1))

`dice_coef` adapted from:
- https://stackoverflow.com/questions/72195156/correct-implementation-of-dice-loss-in-tensorflow-keras
- https://www.kaggle.com/code/shashwatraman/simple-unet-baseline-train-lb-0-580

In [173]:
def dice_coef(y_true, y_pred, threshold=0.5, smooth=0.001):
    y_true_f = backend.flatten(tf.cast(y_true, tf.float32))
    y_pred_f = backend.flatten(y_pred)
    intersection = backend.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (backend.sum(y_true_f) + backend.sum(y_pred_f) + smooth)
    return dice


def dice_coef_loss(y_true, y_pred, smooth):
    return 1 - dice_coef(y_true, y_pred, smooth)

In [ ]:
# Configure the model for training.
# We use the "sparse" version of categorical_crossentropy
# because our target data is integers.
model.compile(optimizer="rmsprop", loss=dice_loss)

callbacks = [
    keras.callbacks.ModelCheckpoint("contrails-unet.h5", save_best_only=True)
]

# Train the model, doing validation at the end of each epoch.
model.fit(train_set, epochs=Config.num_epochs, validation_data=valid_set, callbacks=callbacks)


Adapted from: https://www.kaggle.com/code/shashwatraman/simple-unet-baseline-train-lb-0-580